In [39]:
!pip uninstall numpy

Found existing installation: numpy 1.25.1
Uninstalling numpy-1.25.1:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.11
    /usr/local/lib/python3.11/dist-packages/numpy-1.25.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-7a851222.3.23.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.25.1


In [40]:
!pip install numpy==1.25.1

  Using cached numpy-1.25.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.1 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.25.1 which is incompatible.


In [1]:
import os
from os import listdir
from os.path import isfile, join
import csv
import pandas as pd
import tensorflow as tf

from google.colab import drive
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [72]:
# folder containing the images
image_folder = 'data'
image_csv = 'images.csv'

In [73]:
# collect file paths and names
image_data = []
for root, _, files in os.walk(image_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            file_path = os.path.join(root, file)
            image_data.append([file_path, file])

In [74]:
# Debugging: Check if any images were found
if not image_data:
    print(f"No image files found in the folder: {image_folder}")
else:
    print(f"Found {len(image_data)} image files.")

Found 150 image files.


In [75]:
try:
    # Write to CSV
    with open(image_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['image_path', 'image_name'])  # Header
        writer.writerows(image_data)
    print(f"CSV file created at: {image_csv}")
except Exception as e:
    print(f"Error writing to CSV: {e}")

CSV file created at: images.csv


In [76]:
# Load image paths from CSV
df = pd.read_csv("images.csv")

In [77]:
# Make sure the csv has a column 'image_path'
image_paths = df["image_path"].tolist()

In [78]:
# Load label names (male, female)
label_lines = [line.rstrip() for line in tf.io.gfile.GFile("retrained_labels.txt")]

In [79]:
# Load trained model
with tf.io.gfile.GFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [80]:
# ------------ Model for Age detection --------#
age_weights = "age_deploy.prototxt"
age_config = "age_net.caffemodel"

In [81]:
# Model requirements for image
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
        '(25-32)', '(38-43)', '(48-53)', '(60-100)']
model_mean = (78.4263377603, 87.7689143744, 114.895847746)

In [82]:
# ------------- Model for face detection---------#
face_detector = dlib.get_frontal_face_detector()

In [83]:
# Make predictions
predictions_list = []

In [84]:
with tf.compat.v1.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

    mssg = "age"
    age = 0

    for image_path in image_paths:

        image_data = tf.io.gfile.GFile(image_path, 'rb').read()
        predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})

        # START OF AGE DETECTION CODE

        img = cv2.imread(image_path)
        #print(img)


        img = cv2.resize(img, (720, 640))
        #print(img)

        img.astype('uint8')
        age_Net = cv2.dnn.readNet(age_config, age_weights)

        frame = img.copy()

        # storing the image dimensions
        fH = img.shape[0]
        fW = img.shape[1]

        Boxes = [] # to store the face co-ordinates
        # mssg = 'Face Detected' # to display on image

        # converting to grayscale
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # -------------detecting the faces--------------#
        faces = face_detector(img_gray)

        # If no faces are detected
        if not faces:
            mssg = 'No face detected'
            # cv2.putText(img, f'{mssg}', (40, 40),cv2.FONT_HERSHEY_SIMPLEX, 2, (200), 2)
            # Use cv2_imshow instead of cv2.imshow
            # cv2_imshow(img)
            # cv2.waitKey(0)

        else:
            # --------- Bounding Face ---------#
            for face in faces:
                x = face.left() # extracting the face coordinates
                x = max(0, x)

                y = face.top()
                y = max(0, y)

                x2 = face.right()
                x2 = max(0, x2)

                y2 = face.bottom()
                y2 = max(0, y2)

                # rescaling those coordinates for our image
                box = [x, y, x2, y2]
                Boxes.append(box)
                cv2.rectangle(frame, (x, y), (x2, y2),(00, 200, 200), 2)

            for box in Boxes:

                # TEST!!!

                face = frame[box[1]:box[3], box[0]:box[2]]

                # ----- Image preprocessing --------#
                blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), model_mean, swapRB=False)

                # -------Age Prediction---------#
                age_Net.setInput(blob)
                age_preds = age_Net.forward()
                age = ageList[age_preds[0].argmax()]

                # cv2.putText(frame, f'{mssg}:{age}', (box[0],box[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0, 255, 255), 2, cv2.LINE_AA)
                # Use cv2_imshow instead of cv2.imshow
                # cv2_imshow(frame)
                # cv2.waitKey(0)

                # TEST!!!
                # print(age)

          # This code is modified by Susobhan Akhuli


        # END OF AGE DETECTION CODE


        # Get the most confident prediction
        top_label_index = predictions[0].argsort()[-1]
        predicted_label = label_lines[top_label_index]

        # Store results
        predictions_list.append({"image_path": image_path, "gender": predicted_label, "age": age})

In [85]:
predictions_list[:5]

[{'image_path': 'data/1 (328).jpg', 'gender': 'male', 'age': 0},
 {'image_path': 'data/1 (307).jpg', 'gender': 'male', 'age': 0},
 {'image_path': 'data/1 (399).jpg', 'gender': 'female', 'age': 0},
 {'image_path': 'data/1 (284).jpg', 'gender': 'male', 'age': 0},
 {'image_path': 'data/1 (397).jpg', 'gender': 'male', 'age': '(38-43)'}]

In [86]:
# Save predictions to CSV
results_df = pd.DataFrame(predictions_list)
results_df.to_csv("classified_images.csv", index=False)

In [87]:
results_df.to_csv("/content/drive/My Drive/classified_images.csv", index=False)

In [88]:
!rm -rf data/